In [ ]:
import subprocess
from pathlib import Path
import json
import sys
sys.path.append("..")
from scripts.client import VisAVisClient
from scripts.make_protocol import make_protocol

In [ ]:
folder = '../movies/Channel_6x100/'
folder_path = Path(folder)
folder_path.mkdir(exist_ok=True, parents=True)

channel_height = 6
with open("../parameters.json", 'r') as json_file:
    PARAMETERS_DEFAULT = json.load(json_file)

In [ ]:
client = VisAVisClient(
    sim_root=folder_path
)

protocol_file_path = make_protocol(
    pulse_intervals = [500],
    duration=2,
    out_folder='./'
)

In [ ]:
def extinction_detector(states):
    df = states[states['seconds'] == 320]
    infected_sum = (df['E'] + df['I']).sum()
    return (infected_sum == 0)

def chaos_detector(states):
    df = states[states['seconds'] == 400]
    infected_sum = (df['E'] + df['I']).sum()
    return (infected_sum > 0)

            
def save_with_mp4(folder=folder):
    subprocess.run(f"rm {folder}/movie.mp4", shell=True)
    subprocess.run(f"ffmpeg -framerate 15 -pattern_type glob -i '{folder_path}/{folder}/*.png' -c:v libx264 {folder_path}/{folder}/movie.mp4", shell=True)


In [ ]:
seed = 0
while True:
    seed += 1
    results =  client.run(
        parameters_json=PARAMETERS_DEFAULT,
        protocol_file_path = protocol_file_path,
        seed=seed
        )    
    if chaos_detector(results.states):
        dir_name = f"Simulation{seed}"
        results =  client.run(
            parameters_json=PARAMETERS_DEFAULT,
            protocol_file_path = protocol_file_path,
            seed=seed,
            dir_name=dir_name,
            images=True,
            clean_up=False
            ) 
        save_with_mp4(dir_name)
        break

In [ ]:
subprocess.run("rm out.protocol", shell=True)

## Seed

775098 - chaos for 24x60 channel

493523, 5, 99, 233 - chaos for 6x100 channed

34 - ext for 6x100 channel

let barrier_size: usize = 5;

for h in 0..barrier_size{
    // R barrier
    let ref mut c = lattice.cells[h*Lattice::WIDTH + w];
    c.molecules[Mol::I as usize] = 0;
    c.molecules[Mol::E as usize] = 0;
    c.molecules[Mol::R as usize] = 1;
}